<a href="https://colab.research.google.com/github/shaghayegh5ghasemi/supervised_fuzzy_clustering/blob/main/Supervised_Fuzzy_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import sys

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load data
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=1, shuffle=True, num_workers=0)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# plot the image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy() # convert from tensor
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

Files already downloaded and verified
Files already downloaded and verified


In [3]:
def make_window(img, label):
  windows = img.unfold(1, 3, 2).unfold(2, 3, 2) # unflod(dimension, size, step)
  selected_windows = []
  for n in range(2):
    i = torch.randint(0, 14, (1, 1))
    j = torch.randint(0, 14, (1, 1))
    selected_windows.append([windows[:, i[0, 0], j[0, 0]], label])
  return selected_windows

def make_dataset():
  dataset = []
  for i, (inputs, labels) in enumerate(trainloader):
    dataset = dataset + make_window(inputs[0], labels[0])
    print(sys.getsizeof(dataset))
  return dataset

# extract 3x3 window from 3x36x36 img => we have 12 3x3x3 images made from each individual image in dataset
# width = 32
  # height = 32
  # window_lists = []
  # npimg = img.numpy()
  # print(npimg)
  # row = 0
  # column = 0
  # while(row < width):
  #   if row + 3 > width:
  #     row = width - 3
  #   while(column < height):
  #     if column + 3 > height:
  #       column = height - 3 
  #     mini_tensor = torch.from_numpy(npimg[0:3, row:row+3, column:column+3])
  #     window_lists.append([mini_tensor, label])
  #     column = column + 3
  #   column = 0
  #   row = row + 3

# get some random training images
# dataiter = iter(trainloader)
# images, labels = dataiter.next()

# patches = make_window(images[0], labels[0])

# for i in range(len(patches)):
#   print(patches[i][0].shape)

In [4]:
dataset = make_dataset()

Streaming output truncated to the last 5000 lines.
720088
720104
720120
720136
720152
720168
720184
720200
720216
720232
720248
720264
720280
720296
720312
720328
720344
720360
720376
720392
720408
720424
720440
720456
720472
720488
720504
720520
720536
720552
720568
720584
720600
720616
720632
720648
720664
720680
720696
720712
720728
720744
720760
720776
720792
720808
720824
720840
720856
720872
720888
720904
720920
720936
720952
720968
720984
721000
721016
721032
721048
721064
721080
721096
721112
721128
721144
721160
721176
721192
721208
721224
721240
721256
721272
721288
721304
721320
721336
721352
721368
721384
721400
721416
721432
721448
721464
721480
721496
721512
721528
721544
721560
721576
721592
721608
721624
721640
721656
721672
721688
721704
721720
721736
721752
721768
721784
721800
721816
721832
721848
721864
721880
721896
721912
721928
721944
721960
721976
721992
722008
722024
722040
722056
722072
722088
722104
722120
722136
722152
722168
722184
722200
722216
722232
7222

In [ ]:
imshow(torchvision.utils.make_grid(dataset[120][0]))
print(' '.join('%5s' % classes[dataset[120][1]]))

In [ ]:
# initialize the centroids and the parameters
gamma = 1
sigma = 0.5
trs = 0.5
beta = 1


In [ ]:


# for i in range(100): 
#     if labels[i] == 9: 
#       print(images[i])
#       #imshow(torchvision.utils.make_grid(images[i]))
#       print(' '.join('%5s' % classes[labels[i]]))

imshow(torchvision.utils.make_grid(images[0]))
lists = make_window(img, labels[55])
# print(labels[55])
# imshow(torchvision.utils.make_grid(lists[55][0]))
#print(lists[55][1])
# print(lists[55][0])
# print(len(lists[55][0]))

for i in range(len(lists)): 
  imshow(torchvision.utils.make_grid(lists[i][0]))
  print(' '.join('%5s' % classes[lists[i][1]]))